# Preparation of ChAR-seq files for analysis in python with `chartools`

In this notebook we generate the necessary files to ingest the ChAR-seq data into . The starting point are dna.bed.gz and rna.bed.gz files generated by the snakemake preprocessing pipeline. 

The path of all the snakemake pipeline output files and of the files generated by this script is specific to the Sherlock computer cluster where the analysis was run. Users will need to adjust these paths according to their system. 

Required input files are .rna.bed.gz and .dna.bed.gz for all samples. 
The samples processed here are named as follow:
- ES rep 1: TFONM2_ES
- ES rep 2: TMF2_Rep2B_ES
- DE rep 1: TGNQ5_DE
- DE rep 2: TMF3_Rep2A_DE

## Cleanup multimappers and ambiguous reads


We first filter the rna.bed.gz and dna.bed.gz to remove reads that map to more that one annotated RNA on the RNA side, or with Star alignment score less than 255. We generate separate files for reads mapping to exons, intron on UTLs. On the DNA side, we filter reads with botwie2 alignment score less that 40. 

In [ ]:
%%bash
function clean_up_bed_dna {
     export LC_COLLATE="C"
        fin="${1}.bed.gz" #rna
        fout="${3}"
        if [ "${2}" = "exons" ]; then
            

            zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}(substr($12,4,1)=="T" && $10>254 && $5>39){print $1, $2, $3, $4, $5, $6}'> "${fout}"

        elif [ "${2}" = "introns" ]; then
           

            zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}(substr($12,4,1)=="G" && $10>254 && $5>39){print $1, $2, $3, $4, $5, $6}'  > "${fout}"
        else 
            
            zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}($10>254 && $5>39){print $1, $2, $3, $4, $5, $6}'  > "${fout}"
        fi
}

function clean_up_bed_rna {
     export LC_COLLATE="C"
        fin="${1}.bed.gz" #rna
        fout="${3}"
        if [ "${2}" = "exons" ]; then
            

            zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}($5>254 && substr($12,4,1)=="T" && $10>39){print $1, $2, $3, $4, $5, $6}'> "${fout}"

        elif [ "${2}" = "introns" ]; then
           

            zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}($5>254 && substr($12,4,1)=="G" && $10>39){print $1, $2, $3, $4, $5, $6}'  > "${fout}"
        else 
            
            zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}($5>254 && $10>39){print $1, $2, $3, $4, $5, $6}'  > "${fout}"
        fi
}

export -f clean_up_bed_dna
export -f clean_up_bed_rna

In [ ]:
%%bash
module load system parallel

#First cleanup dna.bed files to remove multimappers (Q<40), and split file based on whether the corresponding RNA side of the reads maps to exons, introns. all captures exons and introns together in the same file.
parallel --dry-run -j8 'clean_up_bed_dna /oak/stanford/groups/astraigh/differentiation_paper_data/charles/libraries/{1}/data/{2}/pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna {3} /oak/stanford/groups/astraigh/differentiation_paper_data/data/chardata/dna.blacklisted.Q255Q40.{2}.{3}.bed' ::: novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun :::+ TFONM2_ES TGNQ5_DE TMF2_Rep2B_ES TMF3_Rep2A_DE ::: all exons introns
                                    
#Repeat with reads where the RNA doesn't map to annotated genes but to UTLs. 
parallel --dry-run -j8 'clean_up_bed_dna /oak/stanford/groups/astraigh/differentiation_paper_data/charles/libraries/{1}/data/{2}/pairs/strngtieCharOnly_hg38/exons/filtered/DNAq15-blacklisted_RNAunq/dna {3} /oak/stanford/groups/astraigh/differentiation_paper_data/data/chardata/dna.blacklisted.Q255Q40.{2}.intergenic.bed' ::: novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun :::+ TFONM2_ES TGNQ5_DE TMF2_Rep2B_ES TMF3_Rep2A_DE ::: all

#cleanup rna.bed files to remove multimappers (STAR Q <255) or reads that can not be unambiguously annotated by a signle gene.  
parallel --dry-run -j8 'clean_up_bed_rna /oak/stanford/groups/astraigh/differentiation_paper_data/charles/libraries/{1}/data/{2}/pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/rna {3} /oak/stanford/groups/astraigh/differentiation_paper_data/data/chardata/rna.blacklisted.Q255Q40.{2}.{3}.bed' ::: novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun :::+ TFONM2_ES TGNQ5_DE TMF2_Rep2B_ES TMF3_Rep2A_DE ::: all exons introns
                                    
#repeat with reads where the RNA doesn't map to annotated genes but to UTLs.                                 
parallel --dry-run -j8 'clean_up_bed_rna /oak/stanford/groups/astraigh/differentiation_paper_data/charles/libraries/{1}/data/{2}/pairs/strngtieCharOnly_hg38/exons/filtered/DNAq15-blacklisted_RNAunq/rna {3} /oak/stanford/groups/astraigh/differentiation_paper_data/data/chardata/rna.blacklisted.Q255Q40.{2}.intergenic.bed' ::: novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun :::+ TFONM2_ES TGNQ5_DE TMF2_Rep2B_ES TMF3_Rep2A_DE ::: all

## Generate .COOcounts files for cis binding RNAs

`rna.Q255Q40.cis.{exons|introns|all}.COOcounts.tsv` contain the mapping coordiate of the RNA side of each read where the DNA contact is on CIS chromosome. The columns are RNA_chr, RNA_3prime_end, ENSGnumber. They are sorted by ENSGnumber then position. 

Requires `rna.bed.gz`.

We make it for each sample using `rna.bed.gz` file located, with respect to the root of the pipeline output for the specific sample, in the following folders
- for the RNA mapping to annotated genes, using the rna.bed.gz file in `pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/`
- for the RNA mapping to UTLs, using the rna.bed.gz file in `pairs/gencondeV29_hg38/strngtieCharOnly_hg38/exons/DNAq15-blacklisted_RNAunq/`


These files are used to model the CIS contacts and run the generative model for producing predicted ChAR-seq maps.  

In [ ]:
%%bash
function make_COO_rnaCIS {
    export LC_COLLATE="C"
    fin="${1}.bed.gz" #rna
    if [ "${2}" = "exons" ]; then
        fout="${1}.Q255Q40.cis.exons.COOcounts.tsv"
    
        zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}($5>254 && substr($12,4,1)=="T" && $18==1 && $10>39){print $1, $2, $17}' | sort -k3,3 -k1,1 -k2,2n > "${fout}"
        
    elif [ "${2}" = "introns" ]; then
        fout="${1}.Q255Q40.cis.introns.COOcounts.tsv"
    
        zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}($5>254 && substr($12,4,1)=="G" && $18==1 && $10>39){print $1, $2, $17}' | sort -k3,3 -k1,1 -k2,2n > "${fout}"
    else 
        fout="${1}.Q255Q40.cis.all.COOcounts.tsv"
        zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}($5>254 && $18==1 && $10>39){print $1, $2, $17}' | sort -k3,3 -k1,1 -k2,2n > "${fout}"
    fi
}


export -f make_COO_rnaCIS

#run for reads where the RNA maps to annotated genes, separately for exons, introns, or both exons and introns together in the same file (all)
parallel --dry-run -j8 'make_COO_rnaCIS /oak/stanford/groups/astraigh/differentiation_paper_data/charles/libraries/{1}/data/{2}/pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/rna {3}' ::: novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun :::+ TFONM2_ES TGNQ5_DE TMF2_Rep2B_ES TMF3_Rep2A_DE ::: exons introns all

# repeat for RNAs mapping to UTLs
parallel --dry-run -j6 'make_COO_rnaCIS /oak/stanford/groups/astraigh/differentiation_paper_data/charles/libraries/{1}/data/{2}/pairs/strngtieCharOnly_hg38/exons/filtered/DNAq15-blacklisted_RNAunq/rna {3}' ::: novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun :::+ TFONM2_ES TGNQ5_DE TMF2_Rep2B_ES TMF3_Rep2A_DE ::: all

## Generate .COOcounts files for DNA

`dna.Q255Q40.{exons|introns|all}.COOcounts.tsv` contains the DNA side mapping position of all contacts from exonic/intronic or total reads. Sorted by ENSG then chr/pos. Of the form
```text
chr1    27566015        ENSG00000000003.14
chr1    27566015        ENSG00000000003.14
chr1    46476066        ENSG00000000003.14
```

Requires `dna.bed.gz`.

We make it for each sample using `dna.bed.gz` file located, with respect to the root of the pipeline output for the specific sample, in the following folders
- for the RNA mapping to annotated genes, using the dna.bed.gz file in `pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/`
- for the RNA mapping to UTLs, using the dna.bed.gz file in `pairs/gencondeV29_hg38/strngtieCharOnly_hg38/exons/DNAq15-blacklisted_RNAunq/`

These files are used to ingest the ChARseq data into `chartable` objects using `chartools`

In [ ]:
function make_COO_dna {
    export LC_COLLATE="C"
    fin="${1}.bed.gz" #dna
    if [ "${2}" = "exons" ]; then
        fout="${1}.Q255Q40.exons.COOcounts.tsv"
    
        zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}(substr($12,4,1)=="T" && $10>254 && $5>39){print $1, $2, $17}' | sort -k3,3 -k1,1 -k2,2n > "${fout}"
        
    elif [ "${2}" = "introns" ]; then
        fout="${1}.Q255Q40.introns.COOcounts.tsv"
    
        zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}(substr($12,4,1)=="G" && $10>254 && $5>39){print $1, $2, $17}' | sort -k3,3 -k1,1 -k2,2n > "${fout}"
    else 
        fout="${1}.Q255Q40.all.COOcounts.tsv"
        zcat "${fin}" | awk -F $'\t' 'BEGIN{OFS=FS}($10>254 && $5>39){print $1, $2, $17}' | sort -k3,3 -k1,1 -k2,2n > "${fout}"
    fi
}

export -f make_COO_dna

parallel -u -t -j8 'make_COO_dna /oak/stanford/groups/astraigh/differentiation_paper_data/charles/libraries/{1}/data/{2}/pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna {3}' ::: novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun :::+ TFONM2_ES TGNQ5_DE TMF2_Rep2B_ES TMF3_Rep2A_DE ::: all exons introns
                                    
parallel --dry-run -j6 'make_COO_dna /oak/stanford/groups/astraigh/differentiation_paper_data/charles/libraries/{1}/data/{2}/pairs/strngtieCharOnly_hg38/exons/filtered/DNAq15-blacklisted_RNAunq/dna {3}' ::: novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun novchar2/CL_2020-05-06_rerun :::+ TFONM2_ES TGNQ5_DE TMF2_Rep2B_ES TMF3_Rep2A_DE ::: all

## Ingest ChARseq data into chartable objects for data handling in python through the `chartools` package

Load necessary packages

In [1]:
from tqdm import tqdm_notebook

In [2]:
import os
import pandas as pd
import pickle
from chartools import charcounter as cc

Define where to find the data files for each samples

In [3]:
#common root folder for the snakemake pipeline runs for various samples
data_root='/oak/stanford/groups/astraigh/differentiation_paper_data/charles/libraries'
# subfolder containing the output files of the snakemake preprocessing pipeline for ES_rep1 (sample name TFONM2_ES), DE_rep1 (sample name TGNQ5_DE), 
NOVA1="novchar1/NOVAseq_06-27-2019/CL_2020-05-06_rerun/data"
# subfolder containing the output files of the snakemake preprocessing pipeline for ES_rep2 (sample name TMF2_Rep2B_ES), DE_rep2 (sample name TMF3_Rep2A_DE),
NOVA2="novchar2/CL_2020-05-06_rerun/data"

NOVA1_smpls=['TFONM2_ES','TGNQ5_DE']
NOVA2_smpls=['TMF2_Rep2B_ES','TMF3_Rep2A_DE']
libs={}
for k in NOVA1_smpls:
    libs[k]=os.path.join(data_root,NOVA1, k)
for k in NOVA2_smpls:
    libs[k]=os.path.join(data_root,NOVA2, k)
    
#list of all samples to process (in order: ES_rep1, DE_rep1, ES_rep2, DE_rep2)
smpls=['TFONM2_ES','TGNQ5_DE','TMF2_Rep2B_ES','TMF3_Rep2A_DE']

Load gene coordinates and chromosome size info

In [5]:
genes=pd.read_csv('/oak/stanford/groups/astraigh/charseq2.0/genomes/hsapiens/grch38_foralign/ENSG_1_X.csv', index_col=0)

chr_df=pd.read_csv('/oak/stanford/groups/astraigh/charseq2.0/genomes/hsapiens/grch38_foralign/resources/chrNameLength_1-22X.txt', names=['chr','len'], sep='\t', index_col=0)

gene_strand_dict={k: (True if v=="-" else False) for k, v in genes.strand.to_dict().items()}


chr_list=list(chr_df.index)
chr_list.sort()
chr_to_chrid={v:i for i,v in enumerate(chr_list)}

gene_to_chrid=genes['chr'].map(chr_to_chrid).to_dict()

for k, v in gene_to_chrid.items():
    gene_to_chrid[k]=[v,v]
    
gene_to_id={k:[i,gene_to_chrid[k][0]] for i, k in enumerate(list(genes.index))}

chr_LUT=[""]*len(chr_to_chrid)
for k, v in chr_to_chrid.items():
    chr_LUT[v]=k
    
chr_len_vec=chr_df.reindex(chr_LUT).values.flatten()

Load coordinates of all UTL genes

In [6]:
intergenes=pd.read_csv('/home/groups/astraigh/differentiation_charseq_paper/notebooks/charles/13_intergenic/data/charOnly/strngtie.1to22X.NOdup.csv', index_col=0)

intergenes_strand_dict={k: (True if v=="-" else False) for k, v in intergenes.strand.to_dict().items()}

intergene_to_chrid=intergenes['chr'].map(chr_to_chrid).to_dict()

for k, v in intergene_to_chrid.items():
    intergene_to_chrid[k]=[v,v]
    
intergene_to_id={k:[i,intergene_to_chrid[k][0]] for i, k in enumerate(list(intergenes.index))}


### Ingest into chartable objects RNA coordinates of the RNA-DNA contact point for each RNA binding to CIS chrosomome.

These objects will be used for building generative model for ChAR-seq data and producing expected contact maps. 

We load the COO files produces above `rna.Q255Q40.cis.{exons|introns|all}.COOcounts.tsv` into a chartable object and pickle it into `rna.Q255.cis.{exons|introns|all}.chartable.{resolution}.pickle`. Once pickled, these files can be reloaded later. 

We do this first ror RNAs mapping to annotated genes. We generate separate chartable objects with just the RNAs mapping to exons or introns. We also generate a "combined" chartable with all the RNAs mapping to annotated genes, regardless if the mapping is to exons or introns.

The pickled chartables are located in:

`pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/rna.Q255Q40.cis.{exons|introns|all}.chartable.{resolution}.pickle`

Resolution is always 1bp for RNA contact points

In [ ]:
for i, s in enumerate(smpls):
    for t in ['all','exons','introns']:
        for j in [1]:
            print((s,t,j))
            rnaCOO=os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/rna.Q255Q40.cis.%s.COOcounts.tsv'%t)
            rnaCOOout=os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/rna.Q255Q40.cis.%s.chartable.%g.pickle'%(t,j))
            tto=cc.Chartable_binned(chr_to_chrid, chr_len_vec, annot_chr_dict=gene_to_id, binsize=j)
            tt=tto.copy()
            tt.load_raw_counts(rnaCOO, reorganize_genes=False, compress=True, compress_at=None)

            with open(rnaCOOout, 'wb') as output:
                pickle.dump(tt, output, pickle.HIGHEST_PROTOCOL)

We repeat this for RNAs mapping to UTLs

In [ ]:
for i, s in enumerate(smpls):
    for t in ['all']:
        for j in [1]: 
            print((s,t,j))
            rnaCOO=os.path.join(libs[s],'pairs/strngtieCharOnly_hg38/exons/filtered/DNAq15-blacklisted_RNAunq/rna.Q255Q40.cis.%s.COOcounts.tsv'%t)
            rnaCOOout=os.path.join(libs[s],'pairs/strngtieCharOnly_hg38/exons/filtered/DNAq15-blacklisted_RNAunq/rna.Q255Q40.cis.%s.chartable.%g.pickle'%(t,j))
            tto=cc.Chartable_binned(chr_to_chrid, chr_len_vec, annot_chr_dict=intergene_to_id, binsize=j)
            tt=tto.copy()
            tt.load_raw_counts(rnaCOO, reorganize_genes=False, compress=True, compress_at=None)

            with open(rnaCOOout, 'wb') as output:
                pickle.dump(tt, output, pickle.HIGHEST_PROTOCOL)

### Ingest into chartable objects the DNA contacts for each gene.

`chartables` stores genomic coordinates in compressed format, where only the genomic coordinates of where contacts are observed (typically DpnII sites) are stored. The loci are indexed and serve as columns in the sparse matrix at the core of the chartable objects. Because contacts may be observed at different subets of genomic loci for each sample, the meaning of the column ID could be different for each sample. Because we don't want that, we don't compress all the chartables independently from one another. Instead, we first make a non-compressed version of these tables, (where each column represents a 10bp interval on the genome) `dna.Q255.{exons|introns|all}.chartable.nocompress.10.pickle`. Then we will next compress the data from all samples together, sot that exons/introns/UTLs chartables for all samples have the same compression scheme (meaning each column represents the same genomic coordinate) 

#### Uncompressed chartables

For the RNAs mapping to annotated genes

In [ ]:
for i, s in enumerate(smpls):
    for t in ['all','exons','introns']:
        for j in [10]: #resolution of the binning 10bp
            print((s,t,j))
            dnaCOO=os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.COOcounts.tsv'%t)
            dnaCOOout=os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.nocompress.%g.pickle'%(t,j))
            tto=cc.Chartable_binned(chr_to_chrid, chr_len_vec, annot_chr_dict=gene_to_id, binsize=j)
            tt=tto.copy()
            tt.load_raw_counts(dnaCOO, reorganize_genes=False, compress=False, compress_at=None)

            with open(dnaCOOout, 'wb') as output:
                pickle.dump(tt, output, pickle.HIGHEST_PROTOCOL)

For RNAs mapping to UTLs

In [ ]:
#intergeninc stuff
for i, s in enumerate(smpls):
    for t in ['all']:
        for j in [10]: #resolution of the binning 10bp
            print((s,t,j))
            dnaCOO=os.path.join(libs[s],'pairs/strngtieCharOnly_hg38/exons/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.COOcounts.tsv'%t)
            print(dnaCOO)
            dnaCOOout=os.path.join(libs[s],'pairs/strngtieCharOnly_hg38/exons/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.nocompress.%g.pickle'%(t,j))
            tto=cc.Chartable_binned(chr_to_chrid, chr_len_vec, annot_chr_dict=intergene_to_id, binsize=j)
            tt=tto.copy()
            tt.load_raw_counts(dnaCOO, reorganize_genes=False, compress=False, compress_at=None)

            with open(dnaCOOout, 'wb') as output:
                pickle.dump(tt, output, pickle.HIGHEST_PROTOCOL)
        

#### Compress all DNA chartables made above using joint support

Now we want to compress the chartable made above for easy loading/handling. We do that by loading the exons/introns and UTLs tables , and then compressing then using the joint support of all three as the compression support. The compressed ones go into --> `dna.Q255.{exons|introns|all}.chartable.compress.10.pickle`

In [ ]:
for j in [10]:
    alldata={}
    for i, s in enumerate(smpls):
        print((j, s))
        dnae=pickle.load(open(os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.nocompress.%g.pickle'%("exons",j)),'rb'))
        dnai=pickle.load(open(os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.nocompress.%g.pickle'%("introns",j)),'rb'))
        dnaa=pickle.load(open(os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.nocompress.%g.pickle'%("all",j)),'rb'))
        
        dnax=pickle.load(open(os.path.join(libs[s],'pairs/strngtieCharOnly_hg38/exons/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.nocompress.%g.pickle'%("all",j)),'rb'))
        hei=dnae.sum(axis=0)
        hi=dnai.sum(axis=0)
        hx=dnax.sum(axis=0)
        ha=dnaa.sum(axis=0)
        #hei.counts=hei.counts+hi.counts+ha.counts #+hx.counts WOOOOO
        
        hei.counts=hei.counts+hi.counts+ha.counts+hx.counts
        hei.compress()
        dnae.compress(x=hei.x.copy())
        dnai.compress(x=hei.x.copy())
        dnax.compress(x=hei.x.copy())
        dnaa.compress(x=hei.x.copy())
        
        
        dnaCOOouta=os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.compress.%g.pickle'%("all",j))
            
        with open(dnaCOOouta, 'wb') as output:
                pickle.dump(dnaa, output, pickle.HIGHEST_PROTOCOL)
        
        dnaCOOoute=os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.compress.%g.pickle'%("exons",j))
            
        with open(dnaCOOoute, 'wb') as output:
                pickle.dump(dnae, output, pickle.HIGHEST_PROTOCOL)
                
        dnaCOOouti=os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.compress.%g.pickle'%("introns",j))
            
        with open(dnaCOOouti, 'wb') as output:
                pickle.dump(dnai, output, pickle.HIGHEST_PROTOCOL)
                
        dnaCOOoutx=os.path.join(libs[s],'pairs/strngtieCharOnly_hg38/exons/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.compress.%g.pickle'%("all",j))
            
        with open(dnaCOOoutx, 'wb') as output:
                pickle.dump(dnax, output, pickle.HIGHEST_PROTOCOL)

## Charseq-ability background

The last thing to do is generate the background model used for correcting the generative models and ChARseq contact maps predictions. To generate the background model, we need to combine the trans-binding profiles of all RNAs that have not been annotated as trans-delocalized (see paper).

We first generate the background model for exonic RNAs.

In [8]:
tscores_exons=pd.read_csv('/home/groups/astraigh/differentiation_charseq_paper/notebooks/charles/17_tscores/data/tscoresAllSamples.exons.tab', sep='\t', header=0)

mrnas_towards_bkg_exons=tscores_exons.loc[(tscores_exons['sample']=='TFONM2') & (tscores_exons['type_coarse']=='protein_coding') &  (tscores_exons['decile']<10)]['ENSG'].to_list()


Now for intronic RNAs

In [9]:
tscores_introns=pd.read_csv('/home/groups/astraigh/differentiation_charseq_paper/notebooks/charles/17_tscores/data/tscoresAllSamples.introns.tab', sep='\t', header=0)

mrnas_towards_bkg_introns=tscores_introns.loc[(tscores_introns['sample']=='TFONM2') & (tscores_introns['type_coarse']=='protein_coding') &  (tscores_introns['decile']<10)]['ENSG'].to_list()

And now for all RNAs combined, regardless of whether they map to exons or introns 

In [10]:
mrnas_towards_bkg_all = list(set(mrnas_towards_bkg_exons+mrnas_towards_bkg_introns))

Now we are ready to make the background model by just loading the compressed DNA chartable and summing up all the signal over the model mRNAs. The output pickled model is -->  `bkg.Q255Q40.{exons|introns|all}.chartable.compress.10.pickle`

In [ ]:
for j in [10]: #resolution 10bp
    alldata={}
    for s in smpls: 
        print((j, s))
        dnae=pickle.load(open(os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.compress.%g.pickle'%("exons",j)),'rb'))
        dnai=pickle.load(open(os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/dna.Q255Q40.%s.chartable.compress.%g.pickle'%("introns",j)),'rb'))
        
    
        
        sname=s.split("_")[0]
        mrnas_towards_bkg_exons=tscores_exons.loc[(tscores_exons['sample']==sname) & (tscores_exons['type_coarse']=='protein_coding') &  (tscores_exons['decile']<10)]['ENSG'].to_list()
        mrnas_towards_bkg_introns=tscores_introns.loc[(tscores_introns['sample']==sname) & (tscores_introns['type_coarse']=='protein_coding') &  (tscores_introns['decile']<10)]['ENSG'].to_list()
        
#         mrnas_towards_bkg_all = list(set(mrnas_towards_bkg_exons+mrnas_towards_bkg_introns))
        
        dnaec, dnaet, _=dnae.cis_trans()
        dnaic, dnait, _=dnai.cis_trans()
        #dnaac, dnaat, _=dnaa.cis_trans()
        
        annote=[k for k in mrnas_towards_bkg_exons if k in dnae.annot_LUT]
        annoti=[k for k in mrnas_towards_bkg_introns if k in dnai.annot_LUT]
       # annota=[k for k in mrnas_towards_bkg_all if k in dnaa.annot_LUT]
        
        Be=dnaet.select_annots(annote).sum(axis=0)
        Bi=dnait.select_annots(annoti).sum(axis=0)
       # Ba=dnaat.select_annots(annota).sum(axis=0)
        
        Bei=Be.copy()
        Bei.counts=Bei.counts+Bi.counts
        
        
        bout=os.path.join(libs[s],'pairs/gencondeV29_hg38/all/filtered/DNAq15-blacklisted_RNAunq/bkg.Q255Q40.chartable.compress.%g.pickle'%j)
            
        with open(bout, 'wb') as output:
            pickle.dump({'exons':Be,'introns':Bi, 'all':Bei}, output, pickle.HIGHEST_PROTOCOL)